In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import *
import tensorflow.keras.backend as K
import seaborn as sns
import pickle
#from src.score import * # load_test_data defined here
from collections import OrderedDict

from tensorflow.keras.layers import Input, UpSampling3D, AveragePooling3D, concatenate, ReLU, Reshape, Concatenate, \
    Permute
#from DLWP.custom import CubeSpherePadding2D, CubeSphereConv2D
from tensorflow.keras.layers import Reshape, Concatenate, Permute
from tensorflow.keras.models import Model
import glob
from tensorflow.keras.models import load_model

import os
import numpy as np

from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation
import cv2
from patchify import * 
#import tensorly as tl

In [3]:
# print("# GPUs Available: ", len(tf.config.experimental.list_physical_devices('XLA_GPU')))
def mse(y, t):
    return np.mean(np.square(y - t))

def psnr(y, t, max_=200.0):
    return 20 * np.log10(max_) - 10 * np.log10(mse(y, t))

def ssim(x, y, max_=200.0):
    mu_x = np.mean(x)
    mu_y = np.mean(y)
    var_x = np.var(x)
    var_y = np.var(y)
    cov = np.mean((x - mu_x) * (y - mu_y))
    c1 = np.square(0.01 * max_)
    c2 = np.square(0.03 * max_)
    return ((2 * mu_x * mu_y + c1) * (2 * cov + c2)) / (
        (mu_x**2 + mu_y**2 + c1) * (var_x + var_y + c2)
    )

In [5]:
data_dir='../data/2020-month/'
file_ = glob.glob(data_dir + "*.nc4")

In [6]:
ds = xr.open_mfdataset(file_, combine="by_coords")

In [7]:
ds_train = ds.sel(time=slice('2020-02-01', '2020-12-01'))
ds_test  = ds.sel(time=slice('2020-01-01', '2020-01-01'))
target = np.squeeze(ds_train.precipitation.values)
maxsd = np.max(target)
print(maxsd)
min_ = 0.0 # min_max.values[0]
max_ = maxsd #min_max.values[1]

1453.403602763257


In [8]:
model = Sequential()
model.add(Conv2D(64, 9, padding="same", input_shape=(20, 20, 1)))
model.add(Activation("relu"))
model.add(Conv2D(32, 1, padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(1, 5, padding="same"))
optimizer = Adam(lr=0.001)
model.compile(optimizer=optimizer, loss="mean_squared_error", metrics=["accuracy"])
weight_filename = "trained_weights.h5"
model.load_weights(os.path.join("../data/model/", weight_filename))

In [48]:
print(ds_train)
data_ = ds_train.precipitation.sel(time=slice("2020-01-01", "2020-01-01")).values
print(data_.shape)
print(data_)

<xarray.Dataset>
Dimensions:        (time: 11, lat: 120, lon: 320)
Coordinates:
  * time           (time) object 2020-02-01 00:00:00 ... 2020-12-01 00:00:00
  * lat            (lat) float64 24.0 24.1 24.2 24.3 ... 35.6 35.7 35.8 35.9
  * lon            (lon) float64 90.0 90.1 90.2 90.3 ... 121.6 121.7 121.8 121.9
Data variables:
    precipitation  (time, lon, lat) float64 dask.array<chunksize=(1, 320, 120), meta=np.ndarray>
(0, 320, 120)
[]


In [10]:
times = [
    "2020-02-01",
    "2020-03-01",
    "2020-04-01",
    "2020-05-01",
    "2020-06-01",
    "2020-07-01",
    "2020-08-01",
    "2020-09-01",
    "2020-10-01",
    "2020-11-01",
    "2020-12-01",
]
for pass_ in range(7, 1000):
    for time in times:
        # Load data
        print("Loading data ", time)
        data_ = ds_train.precipitation.sel(time=slice(time, time)).values

        print("Normalize")
        # Normalize
        data_n = (data_[:, :, :] - min_) / (max_ - min_)

        print("Gen input and output]")
        # Generate input and output
        X = []
        y = []
        for i_ in range(data_n.shape[0]):
            img = data_n[i_, :, :]
            if np.sum(img) > 0:
                scale = 2
                res = cv2.resize(
                    img,
                    dsize=(img.shape[0] * scale, img.shape[1] * scale),
                    interpolation=cv2.INTER_CUBIC,
                )
                res_ = cv2.resize(
                    res,
                    dsize=(
                        int(res.shape[0] * (1 / scale)),
                        int(res.shape[1] * (1 / scale)),
                    ),
                    interpolation=cv2.INTER_CUBIC,
                )
                # data_n_[i_,:,:] = res_
                X.append(img)
                y.append(res_)

        X = np.asarray(X)
        y = np.asarray(y)

        print("Gen sub images")
        # Transform to sub images
        sub_images_ = []
        sub_labels_ = []
        for i_ in range(X.shape[0]):
            image = X[i_]
            patches = patchify(image, (20, 20), step=10)
            sub_images_.append(patches)

            image = y[i_]
            patches = patchify(image, (20, 20), step=10)
            sub_labels_.append(patches)

        sub_images = np.asarray(sub_images_).reshape(-1, 20, 20)
        sub_labels = np.asarray(sub_labels_).reshape(-1, 20, 20)

        X_train = sub_images[:, :, :, np.newaxis]
        y_train = sub_labels[:, :, :, np.newaxis]

        print("Training")
        history = model.fit(
            X_train, y_train, batch_size=64, epochs=100, verbose=1, validation_split=0.1
            # X_train, y_train, batch_size=64, epochs=100, verbose=1, validation_split=0
        )  # , \
        #                         callbacks=[tf.keras.callbacks.EarlyStopping(
        #                             monitor='val_loss',
        #                             min_delta=0,
        #                             patience=2,
        #                             verbose=1,
        #                             mode='auto'
        #                         )])
        model.save_weights(
            os.path.join("../data/model/result/", "srcnn_weight_" + time + str(pass_) + ".hdf5")
        )
        np.save("../data/model/history" + time + "_.npy", history.history)

    # To load
    # history=np.load('my_history.npy',allow_pickle='TRUE').item()
    # https://stackoverflow.com/questions/41061457/keras-how-to-save-the-training-history-attribute-of-the-history-object

Loading data  2020-02-01
Normalize
Gen input and output]
Gen sub images
Training
Epoch 1/100
5/5 [==============================] - 0s 25ms/step - loss: 2.6417e-08 - accuracy: 0.0000e+00 - val_loss: 1.3921e-07 - val_accuracy: 0.0000e+00
Epoch 2/100
5/5 [==============================] - 0s 23ms/step - loss: 2.6508e-08 - accuracy: 0.0000e+00 - val_loss: 1.3963e-07 - val_accuracy: 0.0000e+00
Epoch 3/100
5/5 [==============================] - 0s 19ms/step - loss: 2.6536e-08 - accuracy: 0.0000e+00 - val_loss: 1.3895e-07 - val_accuracy: 0.0000e+00
Epoch 4/100
5/5 [==============================] - 0s 19ms/step - loss: 2.6400e-08 - accuracy: 0.0000e+00 - val_loss: 1.3906e-07 - val_accuracy: 0.0000e+00
Epoch 5/100
5/5 [==============================] - 0s 19ms/step - loss: 2.6384e-08 - accuracy: 0.0000e+00 - val_loss: 1.3914e-07 - val_accuracy: 0.0000e+00
Epoch 6/100
5/5 [==============================] - 0s 18ms/step - loss: 2.6478e-08 - accuracy: 0.0000e+00 - val_loss: 1.3878e-07 - val_accu

KeyboardInterrupt: 